* This notebook executes a clustering procedure on a column of choice in order to identify sub-topics from specific classes on Sports label classes. We are only experimenting with Sports classes to demonstrate the approach that could be used in order to tackle the required task.

* A directly similart approach can be used to several other columns as well. For efficiency reasons, we only experiment with the Sports Column

In [1]:
from sklearn.cluster import OPTICS

* Running Dependencies

In [2]:
%run partA_utilities.ipynb

* Defining Mode and creating directory

In [3]:
mode = "prd"

make_mode_file(mode)

Directory for mode: prd already exists


* Defining Settings

In [4]:
settings = Settings(mode = mode)

* Input Data

In [8]:
input_data = pd.read_csv(settings.train_data_path)
X_train = pd.read_csv(settings.mode_path + '/X_train.csv', index_col=0)
X_val = pd.read_csv(settings.mode_path + '/X_val.csv', index_col=0)
y_train = pd.read_csv(settings.mode_path + '/y_train.csv', index_col=0)


* Create Dataframes of only Sports as label

In [9]:
sports = input_data[input_data['label']=="Sports"].sort_index().reset_index()
X_train = X_train[X_train['original_label_col']=="Sports"].sort_index().reset_index()
X_val = X_val[X_val['original_label_col']=="Sports"].sort_index().reset_index()

* Processing X_train: 
We will use this dataset for demonstration purposes. Similar approach can be used to the whole dataset

In [10]:
X_train = transform_dataframe_list_columns_to_arrays(X_train, 
                                                     list_of_string_cols = ["title", 
                                                                            "content", 
                                                                            "site_title", 
                                                                            "theme"]
                                                    )

* Executing OPTICS Clustering and grabing the labels. Min_samples=3 was selected after experimentation

In [11]:
optics = OPTICS(min_samples=3)
optics.fit(get_array_from_pandas_col_of_arrays(X_train, "content"))
labels = optics.labels_
X_train['clust_labels'] = labels
print(np.unique(labels))

[-1  0  1  2  3  4]


* Merging Dataframes to grab the content and the clustering label

In [12]:
merged_datrame = pd.merge(X_train.loc[:, ["index", "clust_labels"]], 
                          sports.loc[:, ["content", "index", "title"]],
                          on = "index")

* Exploring the produced clusters as the provided labels: Keep in mind that, in OPTICS, the -1 label denotes an un-clustered data point

In [13]:
merged_datrame[merged_datrame['clust_labels']==0] # Makes sence, right???????? Athletics

,index,clust_labels,content,title
127,1477,0,O 17χρονος Ούγγρος Κριστόφ Μίλακ είναι αυτός π...,Παγκόσμιο ρεκόρ και πιο γρήγορος από τον Φελπς!
152,1737,0,Οι Νορβηγοί συνεχίζουν τους αγώνες διοργανώνον...,Καλό 5αρι ο Ίνγκεμπριγκτσεν
158,1826,0,Με αρκετό κρύο και ισχυρό άνεμο ο Μίλτος Τεντό...,Ντάιαμοντ λιγκ Στοκχόλμη: 7.99μ. ο Τεντόγλου
164,1937,0,Μικρή συμμετοχή είχε σήμερα ο αγώνας κλειστού ...,"Μικρή συμμετοχή, καλές επιδόσεις από τους μικρούς"
181,2174,0,Το δεύτερη μίτινγκ κλειστού στίβου της σειρά w...,Δυνατό τεστ και ευκαιρία για όρια


In [14]:
merged_datrame[merged_datrame['clust_labels']==1] # Makes sence, right???????? FORMULA1

,index,clust_labels,content,title
17,213,1,Ο κλασικός βρετανικός (και το καλοκαίρι) καιρό...,O Hamilton πρόλαβε τη βροχή στο FP3
59,664,1,Με το δεξί ξεκίνησε ο Lewis Hamilton την προ...,GP Τουρκίας: Ταχύτερος ο Hamilton στο FP1
118,1394,1,"Με συννεφιά, κρύο και βροχή υποδέχτηκε το Red ...",GP Αυστρίας: Ταχύτερες οι Mercedes στα ελεύθερα
129,1497,1,Μόλις μία ημέρα αγωνιστικής δράσης απέχει από ...,WRC Ράλι Ισπανίας: Αγκαλιά με νίκη-τίτλου ο Ogier
165,1942,1,Μπορεί το απόλυτο ρεκόρ της πίστας του Χανγκαρ...,Video: Ο ταχύτερος γύρος στην ιστορία του Χανγ...
177,2122,1,Η πρώτη και η δεύτερη περίοδος των ελεύθερων δ...,«Άπιαστη» η Mercedes με 1-2 στο FP1 και το FP2


In [15]:
merged_datrame[merged_datrame['clust_labels']==2] # Makes sence, right???????? BASKET

,index,clust_labels,content,title
7,108,2,Ο Παναθηναϊκός είναι έτοιμος για το ματς με τη...,Παπαγιάννης και Σαντ Ρος ενόψει Χίμκι: «Να είμ...
81,865,2,Σόουζα στο Onsports: «Έζησα ξεχωριστές στιγμές...,Σόουζα στο Onsports: «Έζησα ξεχωριστές στιγμές...
100,1209,2,Ο προπονητής του Βελγίου Ρομπέρτο Μαρτίνεθ είχ...,Ρ.Μαρτίνεθ: «Η ομάδα μου το άξιζε»
130,1512,2,Ο Γιαννούλης Λαρεντζάκης επιστρέφει μετά από α...,Λαρεντζάκης: «Σταθεροί στο πλάνο μας...»
192,2376,2,"Στην ήττα από την Αρμάνι Μιλάνο, στον αγώνα κό...",Μάρτιν: «Πρέπει να παίξω με περισσότερη αυτοπε...


In [16]:
merged_datrame[merged_datrame['clust_labels']==3]  # Makes sence, right???????? POLO

,index,clust_labels,content,title
142,1687,3,Με το απόλυτο ολοκλήρωσε τις υποχρεώσεις του γ...,Χριστουγεννιάτικος… περίπατος για Ολυμπιακό - ...
154,1785,3,H ομάδα πόλο του ΑΝΟ Γλυφάδας στους άνδρες θα ...,Τελικό μετά από 25 χρόνια ο ΑΝΟΓ!
155,1794,3,H ομάδα του ΝΟ Χίου είναι αυτή που κλέβει την ...,Εντυπωσιάζει ο ΝΟ Χίου


In [17]:
merged_datrame[merged_datrame['clust_labels']==4] # Makes sence, right???????? VOLLEY

,index,clust_labels,content,title
43,470,4,Ο Φοίνικας Σύρου παρότι βρέθηκε να χάνει με ...,Ο Φοίνικας με ανατροπή 3-2 τον Παναθηναϊκό και...
57,646,4,Δύο στα δύο για την Εθνική κόντρα στην Ουκρανί...,Εθνική ανδρών: Νέα νίκη με 3-1 επί της Ουκρανίας
78,847,4,Οι ξένες της Σαντορίνης είχαν 9/50 επιθέσεις (...,Σούπερ Πανναξιακός πέρασε στο φάιναλ 4
128,1484,4,"Με ηγέτη τον εκπληκτικό Σταύρο Κασαμπαλή, ο οπ...","Βόλεϊ λιγκ, ΑΟΠ Κηφισιάς-ΠΑΟΚ 3-2: Ανατροπή με..."
157,1816,4,Η Τουρκία με τη δύναμη της έδρας νίκησε με 3-1...,Ευρωβόλεϊ γυναικών: Τουρκία-Σερβία ο τελικός


* Unfortunately, most of the sports were unable to be clustered further. However, this was only a quick and dirty approach. Further experimentation could reveal more trends!